## Import Packages and Set Directory

In [138]:
import pandas as pd
import numpy as np
import sys
import argparse
import os
os.chdir('/Users/abc6435/Desktop/RABvcfs')

## Import Data

In [139]:
#Extract Header 
def read_vcf(path):
    with open(path) as file:
        for line in file:
            if line.startswith('#CHROM'):
                return line.lstrip("#").strip().split("\t")
            
cols = read_vcf('/Users/abc6435/Desktop/RABvcfs/rab_chr29_test.vcf')

#Import Data
data = pd.read_csv("rab_chr29_test.vcf", sep='\t', comment='#', names=cols)
pops = pd.read_csv("pops.txt", sep="\t", header=None)

## Parse Data

In [141]:
#Assign Population
popA = pops[pops[0]=="popA"][1].astype(str)
popB = pops[pops[0]=="popB"][1].astype(str)
                             
#Extract Loci
loci = data[['CHROM','POS','REF','ALT']]

#Parse Data
samples = data.drop(columns=['CHROM', 'POS', 'REF', 'ALT','ID','QUAL','FILTER','INFO','FORMAT'])
samples = samples.apply(lambda col: col.str.split(":").str[0])

#Concatenate Dateframes
genotypes = pd.concat([loci,samples], axis=1)

#Add Counter Columns
genotypes['popA_0'] = 0
genotypes['popA_1'] = 0
genotypes['popB_0'] = 0
genotypes['popB_1'] = 0

## Count Genotypes 

In [142]:
#Population A
for sample in popA:
    for row in range(len(genotypes)):
        if genotypes.loc[row, sample] == '0/0' or genotypes.loc[row, sample] == '0|0':
            genotypes.loc[row, 'popA_0'] += 2
        if genotypes.loc[row, sample] == '1/1' or genotypes.loc[row, sample] == '1|1':
            genotypes.loc[row, 'popA_1'] += 2
        if genotypes.loc[row, sample] == '0/1' or genotypes.loc[row, sample] == '0|1':
            genotypes.loc[row, 'popA_0'] += 1
            genotypes.loc[row, 'popA_1'] += 1
        if genotypes.loc[row, sample] == '1/0' or genotypes.loc[row, sample] == '1|0':
            genotypes.loc[row, 'popA_0'] += 1
            genotypes.loc[row, 'popA_1'] += 1

#Population B
for sample in popB:
    for row in range(len(genotypes)):
        if genotypes.loc[row, sample] == '0/0' or genotypes.loc[row, sample] == '0|0':
            genotypes.loc[row, 'popB_0'] += 2
        if genotypes.loc[row, sample] == '1/1' or genotypes.loc[row, sample] == '1|1':
            genotypes.loc[row, 'popB_1'] += 2
        if genotypes.loc[row, sample] == '0/1' or genotypes.loc[row, sample] == '0|1':
            genotypes.loc[row, 'popB_0'] += 1
            genotypes.loc[row, 'popB_1'] += 1
        if genotypes.loc[row, sample] == '1/0' or genotypes.loc[row, sample] == '1|0':
            genotypes.loc[row, 'popB_0'] += 1
            genotypes.loc[row, 'popB_1'] += 1

## Calculate Frequencies


In [143]:
#Population A
genotypes['f_popA_0'] = genotypes['popA_0']/(genotypes['popA_0'] + genotypes['popA_1'])
genotypes['f_popA_1'] = genotypes['popA_1']/(genotypes['popA_0'] + genotypes['popA_1'])

#Population B
genotypes['f_popB_0'] = genotypes['popB_0']/(genotypes['popB_0'] + genotypes['popB_1'])
genotypes['f_popB_1'] = genotypes['popB_1']/(genotypes['popB_0'] + genotypes['popB_1'])



In [145]:
genotypes

,CHROM,POS,REF,ALT,183194841,183194861,183195304,183195312,183195321,183195326,...,507264,507265,popA_0,popA_1,popB_0,popB_1,f_popA_0,f_popA_1,f_popB_0,f_popB_1
0,chr29,270,C,T,./.,./.,./.,./.,./.,./.,...,0/1,./.,3,1,0,0,0.75,0.25,NaN,NaN
1,chr29,309,C,T,./.,./.,./.,./.,./.,./.,...,./.,./.,0,2,0,0,0.00,1.00,NaN,NaN
2,chr29,375,G,A,./.,./.,./.,./.,./.,./.,...,./.,./.,0,2,0,0,0.00,1.00,NaN,NaN
3,chr29,1208,G,A,./.,./.,0/1,./.,./.,./.,...,1/1,./.,1,3,1,1,0.25,0.75,0.5,0.5
4,chr29,1699,A,T,./.,./.,0/1,./.,./.,./.,...,./.,./.,0,0,1,1,NaN,NaN,0.5,0.5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3778,chr29,962252,G,A,0|1,./.,./.,./.,./.,./.,...,0/0,0|1,3,1,1,1,0.75,0.25,0.5,0.5
3779,chr29,962338,G,A,./.,./.,./.,./.,./.,./.,...,0/0,0|1,3,1,0,0,0.75,0.25,NaN,NaN
3780,chr29,962394,G,T,./.,./.,./.,./.,./.,./.,...,0/1,./.,1,1,0,0,0.50,0.50,NaN,NaN
3781,chr29,962524,G,C,./.,./.,./.,./.,./.,./.,...,1/1,./.,0,2,0,0,0.00,1.00,NaN,NaN
